In [1]:
import numpy as np
from numpy import round as r
import numpy.linalg as linalg

In [2]:
# np.set_printoptions(formatter={'float_kind': lambda x: str(np.round(x, 4))})
np.set_printoptions(formatter={'float_kind': "{:.4f}".format})

In [3]:
A = np.array([[9, -5, -6, 3], [1, -7, 1, 0], [3, -4, 9, 0], [6, -1, 9, 8]])
B = np.array([[-8], [38], [47], [-8]])
A, B

(array([[ 9, -5, -6,  3],
        [ 1, -7,  1,  0],
        [ 3, -4,  9,  0],
        [ 6, -1,  9,  8]]),
 array([[-8],
        [38],
        [47],
        [-8]]))

In [4]:
#det + inv
det = r(linalg.det(A), 4)
inv = linalg.inv(A)

print('det(A) = {}\n-----------\nA^(-1) = \n{}'.format(det, inv))

det(A) = -4239.0
-----------
A^(-1) = 
[[0.1113 -0.1493 0.1326 -0.0418]
 [0.0113 -0.1677 0.0304 -0.0042]
 [-0.0321 -0.0248 0.0804 0.0120]
 [-0.0460 0.1189 -0.1861 0.1423]]


In [ ]:
#norms
norm_euc = r(np.linalg.norm(A, 'fro'), 4)
norm_col = np.linalg.norm(A, 1)
norm_row = np.linalg.norm(A, np.inf)
norm_max = 4 * max(list(map(max,  [np.abs(row) for row in A])))

print('euc = {}\ncol = {}\nrow = {}\nmax = {}'.format(norm_euc, norm_col, norm_row, norm_max))

euc = 22.1359
col = 25.0
row = 24.0
max = 36


In [6]:
#cond
cond = r(linalg.norm(A, 1) * linalg.norm(linalg.inv(A), 1), 4)
print('cond = {}'.format(cond))

cond = 11.518


In [ ]:
#gaus
AB = np.hstack([A.astype(float), B.astype(float)])
for (row_n, row) in enumerate(AB):
    divider = row[row_n]
    for lower_row in AB[row_n+1:]:
        factor = lower_row[row_n]/divider
        lower_row -= factor*row
print('Modified matrix:\n{}'.format(AB))
x = np.array([r(AB[-1, -1] / AB[-1, -2])])
for row_n in range(len(A)-2, -1, -1):
    # print(AB[row_n, row_n+1:-1])
    new_x = (AB[row_n, -1] - np.dot(AB[row_n, row_n+1:-1], x))/AB[row_n, row_n]
    x = np.concat([[new_x], x])
print('X-vector:\n{}'.format(x))

Modified matrix:
[[9.0000 -5.0000 -6.0000 3.0000 -8.0000]
 [0.0000 -6.4444 1.6667 -0.3333 38.8889]
 [0.0000 0.0000 10.3966 -0.8793 35.5862]
 [0.0000 0.0000 0.0000 7.0299 -35.1493]]
X-vector:
[0.0000 -5.0000 3.0000 -5.0000]


In [ ]:
#eig vals
print('Eigenvalues: \n{}'.format(r(linalg.eigvals(A), 4)))

Eigenvalues: 
[-6.3888+0.j     12.4965+0.j      6.4461+3.3974j  6.4461-3.3974j]


In [9]:
#jacobi + seidel
diag_dom = list()
for i in range(len(A)):
    row = A[i]
    el = np.abs(row[i])
    row = sum(np.abs(np.delete(row, i)))
    diag_dom.append(bool(el > row))
    
diag_dom1 = list()
for i in range(len(A)):
    col = A[:,  i]
    el = np.abs(col[i])
    col = sum(np.abs(np.delete(col, i)))
    diag_dom1.append(bool(el > col))

symmetrical = np.array_equal(A, A.transpose())

d, l, u = np.diag(np.diag(A)), np.tril(A, k=-1), np.triu(A, k=1)
g = -linalg.inv(d+l) @ u
eig = linalg.eigvals(g)
spectral_radius = r(np.max(np.abs(eig)), 4)
print('Diagonal domination (row): {}\nDiagonal domination (col): {}\nIs symmetrical: {}\nSpectral radius: {}'.format(all(diag_dom), all(diag_dom1), symmetrical, spectral_radius))
    

Diagonal domination (row): False
Diagonal domination (col): False
Is symmetrical: False
Spectral radius: 0.2009


In [13]:
posible_w = list()
for w in range(1, 20):
    T = linalg.inv(d+w/10*l) @ ((1-w/10)*d-w/10*u)
    spectral_radius = r(np.max(np.abs(linalg.eigvals(T))), 4)
    if spectral_radius < 1:
        posible_w.append(w/10)

posible_w

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]

In [28]:
def sor(A, b, omega=1.1, max_iter=100, epsilon=1e-6):
    n = len(b)
    x = np.zeros(n)
    for _ in range(max_iter):
        x_new = np.copy(x)
        for i in range(n):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (1 - omega) * x[i] + omega * (b[i] - s1 - s2) / A[i, i]
        if np.linalg.norm(x_new - x) < epsilon:
            break
        x = x_new
    return x

for w in posible_w:
    print(sor(A, np.array([-8, 38, 47, -8]), omega=w))

[0.0047 -4.9986 2.9979 -5.0049]
[-0.0000 -5.0000 3.0000 -5.0000]
[-0.0000 -5.0000 3.0000 -5.0000]
[-0.0000 -5.0000 3.0000 -5.0000]
[-0.0000 -5.0000 3.0000 -5.0000]
[-0.0000 -5.0000 3.0000 -5.0000]
[-0.0000 -5.0000 3.0000 -5.0000]
[-0.0000 -5.0000 3.0000 -5.0000]
[-0.0000 -5.0000 3.0000 -5.0000]
[0.0000 -5.0000 3.0000 -5.0000]
[-0.0000 -5.0000 3.0000 -5.0000]
[-0.0000 -5.0000 3.0000 -5.0000]
[-0.0000 -5.0000 3.0000 -5.0000]
[-0.0000 -5.0000 3.0000 -5.0000]
[0.0000 -5.0000 3.0000 -5.0000]
